In [1]:
import psycopg2

In [3]:
# establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname = "dw_cs", 
    user = "postgres", 
    host= 'localhost',
    password = "Mu34zi72",
    port = 5432
)

In [4]:
with conn.cursor() as cur:
    
    cur.execute("SELECT * FROM region;")

    rows = cur.fetchall()

    for row in rows:
        print(row)

(0, 'AFRICA                   ', 'lar deposits. blithely final packages cajole. regular waters are final requests. regular accounts are according to ')
(1, 'AMERICA                  ', 'hs use ironic, even requests. s')
(2, 'ASIA                     ', 'ges. thinly even pinto beans ca')
(3, 'EUROPE                   ', 'ly final courts cajole furiously final excuse')
(4, 'MIDDLE EAST              ', 'uickly special accounts cajole carefully blithely close requests. carefully final asymptotes haggle furiousl')


In [5]:
with conn.cursor() as cur:
    
    cur.execute("SELECT * FROM nation;")

    rows = cur.fetchall()

    for row in rows:
        print(row)

(0, 'ALGERIA                  ', 0, ' haggle. carefully final deposits detect slyly agai')
(1, 'ARGENTINA                ', 1, 'al foxes promise slyly according to the regular accounts. bold requests alon')
(2, 'BRAZIL                   ', 1, 'y alongside of the pending deposits. carefully special packages are about the ironic forges. slyly special ')
(3, 'CANADA                   ', 1, 'eas hang ironic, silent packages. slyly regular packages are furiously over the tithes. fluffily bold')
(4, 'EGYPT                    ', 4, 'y above the carefully unusual theodolites. final dugouts are quickly across the furiously regular d')
(5, 'ETHIOPIA                 ', 0, 'ven packages wake quickly. regu')
(6, 'FRANCE                   ', 3, 'refully final requests. regular, ironi')
(7, 'GERMANY                  ', 3, 'l platelets. regular accounts x-ray: unusual, regular acco')
(8, 'INDIA                    ', 2, 'ss excuses cajole slyly across the packages. deposits print aroun')
(9, 'INDONESIA

In [12]:
conn.rollback()
with conn.cursor() as cur:

    for attribute in list("o_orderkey", "o_custkey", "o_orederstatus", "o_totalprice", "o_orderdate", "o_orederpriority", "o_clerk", "o_shippriority", "o_comment"):
    
        cur.execute("SELECT COUNT(*) FROM orders;")

        all_rows = cur.fetchall()

        cur.execute(" SELECT COUNT (DISTINCT o_orderkey) FROM orders;")

        distinct_rows = cur.fetchall()

        cur.execute(" SELECT MAX (o_orderkey) FROM orders;")
        rows = cur.fetchall()
        print(rows)

        cur.execute(" SELECT MIN (o_orderkey) FROM orders;")
        rows = cur.fetchall()




    
    print(rows)

[(15000000,)]
[(60000000,)]
